In [1]:
#Imports
import matplotlib as plt
import pandas as pd
import numpy as np
import re


In [2]:
#Global Definitions
RegexToRemove = r'\(|\)|,|\-|\/|\"|\[|\]'
WordsToRemove = ["de", "en", "con", "y", "para", "x", "la", "el", "a", "no", "un", "dos",
                 "tipo", "sin", "pro", "los", "las", "que", "por", "and", "se", "es", "al",
                 "del", "m", "gb", "d", "va", "ns", "dd", "ddr", "o", "s", "f"]
WordToReplace = ["disco duro", "monitor de color", "carro transportador de libros", "reproductor de discos compactos", "aire acondicionado"]
Replacement = ["disco_duro", "monitor_de_color", "carro_transportador_de_libros", "reproductor_de_discos_compactos", "aire_acondicionado"]
Materiales = ["madera", "metal", "metalica", "metalico", "acero inoxidable", "aluminio", "corcho", "tela"]
Marcas = ["pentium", "hp", "dell", "intel", "sony"]
Colores = ["negro", "verde", "cafe", "azul", ""]
NonProducts = ["Color", "c"]

In [3]:
#Fuctions
def GetDfInfo(Df):
    WordCountDict = dict()
    Entries = list()
    for Row in Df.values:
        Text = [Value + " " for Value in Row[1:] if type(Value) == str]
        Text = "".join(Text).lower()
        Text = re.sub(RegexToRemove, " ", Text)
        for i, _Word in enumerate(WordToReplace):
            if _Word in Text and Text.find(_Word) <= len(_Word):
                Text = Text.replace(_Word, Replacement[i])
        Entries.append(Text)
        Words = list(filter(lambda x: x not in WordsToRemove, Text.split(" ")))
        for Word in Words:
            if Word == "":
                continue
            if Word in WordCountDict:
                WordCountDict[Word] += 1
            else:
                WordCountDict[Word] = 1
    return WordCountDict, np.array(Entries).reshape(len(Entries), 1)

def FixDfEntries(Entries):
    FixeEntries = {"Tipo": [],
             "Producto": [],
             "Marca": [],
             "Color": [],
             "Material": [],
             "Memoria": [],
             "Velocidad": [],
             "Datos_Adicionales": []
             }
    Size = len(Entries)
    for i, Entry in enumerate(Entries):
        if i == 3642:
            pass
        if not Entry or len(Entry[0].split(" ")) < 3:
            continue
        Tipo, Entry = Entry[0].split(" ", 1)

        Producto, Entry = Entry.split(" ", 1)
        if Tipo == "libro":
            Producto = Producto + " " + Entry
        for NonProduct in NonProducts:
            if Producto == NonProduct:
                continue
        if not Producto or re.match("\d+.*\d*", Producto):
            continue

        Marca = re.findall(r"marca [a-z\d]+", Entry)
        Marca = Marca[0] if Marca else "N/A"
        if Marca == "N/A":
            for _Marca in Marcas:
                if _Marca in Entry:
                    Marca = _Marca
                    break

        Entry = Entry.replace(Marca, " ")
        if Marca != "N/A":
            Marca = Marca.split(" ")[1] if Marca.find(" ") > 0 else Marca

        Color = re.findall(r"color [a-zÀ-ÿ]+", Entry)
        Color = Color[0] if Color else "N/A"
        Entry = Entry.replace(Color, " ")
        if Color != "N/A":
            Color = Color.split(" ")[1]

        Material = "N/A"
        for _Material in Materiales:
            if _Material in Entry:
                Material = _Material
                break
        Entry = Entry.replace(Material, " ")

        Memoria = re.findall(r"\d+ ?gb", Entry)
        Memoria = Memoria[0] if Memoria else "N/A"
        FixeEntries["Memoria"].append(Memoria)
        Entry = Entry.replace(Memoria, " ")

        Velocidad = re.findall(r"\d+.?\d* ?ghz", Entry)
        Velocidad = Velocidad[0] if Velocidad else "N/A"
        Entry = Entry.replace(Velocidad, " ")

        FixeEntries["Tipo"].append(Tipo)
        FixeEntries["Producto"].append(Producto)
        FixeEntries["Marca"].append(Marca)
        FixeEntries["Color"].append(Color)
        FixeEntries["Material"].append(Material)
        FixeEntries["Velocidad"].append(Velocidad)
        FixeEntries["Datos_Adicionales"].append(Entry)
    return pd.DataFrame(FixeEntries)

In [5]:
#Data Processing
DataSet = pd.read_excel("Actividad - 10,000.xlsx")

WordCount, Entries = GetDfInfo(DataSet)
WordCount = list(WordCount.items())
WordCount = np.array(WordCount, dtype=[("Word", "U10"), ("Count", "i4")])
WordCount = np.sort(WordCount, order="Count")[::-1]
WordCount

NewDf = FixDfEntries(Entries)
NewDf

,Tipo,Producto,Marca,Color,Material,Memoria,Velocidad,Datos_Adicionales
0,mobiliario,pintarron,alfra,N/A,N/A,N/A,N/A,60x90 alta 20 ene 1999 .
1,equipo,monitor_de_color,hp,N/A,N/A,N/A,N/A,de 15 n.d. 017 2002 alta 07 octubre 2002
2,mobiliario,cesto,N/A,N/A,aluminio,N/A,N/A,de basura n.d. 007 2005 alta 31 agosto 2005
3,transporte,automovil,N/A,N/A,N/A,N/A,N/A,ford escort 1998 placas: hzg1129 no. serie: ...
4,equipo,trituradora,N/A,N/A,N/A,N/A,N/A,de documentos. alta 13 sep 2006 n.d. 07 2006
...,...,...,...,...,...,...,...,...
9786,equipo,computadora,N/A,N/A,N/A,N/A,N/A,según factura 729 de vicente de la cruz herrera
9787,equipo,pintarron,alfra,blanco,N/A,N/A,N/A,modelo 4840 120x240 sin numero de serie
9788,equipo,regulador,N/A,azul,N/A,N/A,N/A,de voltaje industronic 15kva blanco.
9789,equipo,tp,N/A,N/A,N/A,N/A,N/A,link 1000base lx sfp optic smf mini gbic module
